In [3]:
from dataframe_utils import data_get, vars_filter, vars_merge, vars_fill_nan
from workspace import workspace, paths_get, paths_join

## 01 Data Cleaning

For the whole dataset, then split into NAION and ODD separately

In [4]:
workspace('ocumet')
data = data_get('1209-raw.csv', lowercase=True)

print('Raw dataframe size: (%s, %s)' % data.shape)

data = data.dropna(subset=['subject_id'], axis='rows')
data = vars_filter(data, ['rimprof'])
data['bilateral_str'] = data['bilateral'].map({'Y':'bilateral','N':'unilateral'})
data['bilateral_str'] = data['bilateral_str'].fillna('')
data['subject_id'] = data['subject_id'].astype(int)
data = vars_merge(data, {'id':['subject_id', 'eye'], 'group':['diagnosis', 'bilateral_str']})
data['group'] = data['group'].str.replace('_',' ').str.strip()

diagnosis_map = {
    'not control':'Not control', 'ODD-AION':'ODD-NAION'}
for k, v in diagnosis_map.items():
    data.loc[data['group'].str.contains(k, case=False), 'group'] = v

display(data.iloc[:5,:10].head())

print('Clean dataframe size: (%s, %s)' % data.shape)
print(', '.join(sorted(data.group.unique().tolist())))

paths order: b.d.o.t.s
you're in /Users/miaomiaoyu/workspace/ocumet
Raw dataframe size: (192, 86)


,visit_no,date,subject_id,time_since_onset/diagnosis_(months),date_of_image,months,diagnosis,eye,odd-aion_,bilateral
0,5.0,5/6/22 0:00,9,3/1/20,5/6/22 0:00,26.0,NAION,OD,NaN,Y
1,5.0,5/6/22 0:00,9,3/25/22,5/6/22 0:00,1.0,NAION,OS,NaN,Y
2,1.0,6/9/21 0:00,40,1/9/20,6/9/21 0:00,17.0,NAION,OS,NaN,N
3,1.0,10/6/21 0:00,102,9/18/20,10/6/21 0:00,12.0,NAION,OD,NaN,Y
4,1.0,5/13/22 0:00,137,4/8/16,5/13/22 0:00,73.0,NAION,OD,NaN,Y


Clean dataframe size: (183, 65)
Control, NAION, NAION bilateral, NAION unilateral, Not control, ODD, ODD-NAION


In [7]:
independent_vars = 'group'
identification_vars = ['id', 'months', 'visit_no']
dependent_vars = [d_var for d_var in data.columns for this_str in ['mean', 'oct', 'fpf', 'vf', 'hvf'] if d_var.startswith(this_str)]
data_variables = [independent_vars] + identification_vars + dependent_vars

# neaten up the dependent var column names for str.split later
new_dependent_vars = [var.replace('cq_mac_', 'cq_').replace('gcc_mac_', 'gcc_') for var in dependent_vars]
data = data.rename( columns=dict(zip(dependent_vars,new_dependent_vars)) )

dependent_vars = new_dependent_vars

In [8]:
_,d,_,_,_ = paths_get('ocumet')

for x in ['NAION', 'ODD']:
    this_group = [y for y in data['group'].unique().tolist() if x in y]
    group_data = data[data['group'].isin(this_group+['Control'])]
    group_data = vars_fill_nan(group_data, d_vars=dependent_vars, i_vars=independent_vars)

    group_data[data_variables].to_csv(paths_join(d,'1209-%s.csv' % x.lower()), index=False)
    print("%s data saved out as '1209-%s.csv'" % (x, x.lower()))

NAION data saved out as '1209-naion.csv'
ODD data saved out as '1209-odd.csv'
